In [1]:
!pip install -q ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.1 MB/s eta 0:00:00


In [2]:
import json
import textwrap
from ultralytics import YOLO
from pathlib import Path
import yaml
import os
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Source directories
src_train = "/kaggle/input/seadronessee-mot-sampled-dataset/images/train"
src_val = "/kaggle/input/seadronessee-mot-sampled-dataset/images/val"

# Destination directory
output_dir = "/kaggle/working/seadronessee_dataset/images"

# Create output folder if not exists
os.makedirs(output_dir, exist_ok=True)

# Copy the whole folders
shutil.copytree(src_train, os.path.join(output_dir, "train"), dirs_exist_ok=True)
shutil.copytree(src_val, os.path.join(output_dir, "val"), dirs_exist_ok=True)

print("✅ train and val folders copied successfully!")

✅ train and val folders copied successfully!


In [4]:
# Source directories
src_train = "/kaggle/input/seadronessee-mot-sampled-dataset/labels/train"
src_val = "/kaggle/input/seadronessee-mot-sampled-dataset/labels/val"

# Destination directory
output_dir = "/kaggle/working/seadronessee_dataset/labels"

# Create output folder if not exists
os.makedirs(output_dir, exist_ok=True)

# Copy the whole folders
shutil.copytree(src_train, os.path.join(output_dir, "train"), dirs_exist_ok=True)
shutil.copytree(src_val, os.path.join(output_dir, "val"), dirs_exist_ok=True)

print("✅ train and val labels copied successfully!")

✅ train and val labels copied successfully!


In [5]:
mkdir /kaggle/working/cfg

In [6]:
MODEL_YAML = "/kaggle/working/cfg/yolov8_sea.yaml"    # your modified architecture file
DATA_YAML  = "/kaggle/working/cfg/seadronessee.yaml"

In [7]:
dict_file = {
    'train': '/kaggle/working/seadronessee_dataset/images/train',
    'val': '/kaggle/working/seadronessee_dataset/images/val',
    'nc': 3,
    'names': ['swimmer', 'swimmer with life jacket', 'boat']
}

with open(DATA_YAML, 'w') as file:
    yaml.dump(dict_file, file)

In [8]:
yolov8l_sea_text = textwrap.dedent("""\
# YOLOv8-L with SeaDronesSee small-object refinement (YOLO-SR)

nc: 3  # number of classes (swimmer, swimmer_lifejacket, boat)
scales:
  # for 'l' we match YOLOv8l width/depth; we only use this single scale here
  l: [1.0, 1.0, 512]

# Backbone (copied from YOLOv8L style, simplified from generic yolov8.yaml)
backbone:
  - [-1, 1, Conv, [64, 3, 2]]      # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]     # 1-P2/4
  - [-1, 6, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]     # 3-P3/8
  - [-1, 12, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]     # 5-P4/16
  - [-1, 12, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]    # 7-P5/32
  - [-1, 6, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]       # 9

# Head
head:
  # P5 -> P4
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]   # 10
  - [[-1, 6], 1, Concat, [1]]                    # cat P4 backbone
  - [-1, 6, C2f, [512]]                          # 12

  # P4 -> P3
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]   # 13
  - [[-1, 4], 1, Concat, [1]]                    # cat P3 backbone
  - [-1, 6, C2f, [256]]                          # 15 (P3 base feature)

  # --- NEW small-object refinement block on P3 ---
  - [-1, 1, Conv, [256, 3, 1]]                   # 16 refine1
  - [-1, 1, Conv, [256, 3, 1]]                   # 17 refine2 (P3_refined)

  # Down to P4 (from refined P3)
  - [-1, 1, Conv, [256, 3, 2]]                   # 18
  - [[-1, 12], 1, Concat, [1]]                   # cat head P4
  - [-1, 6, C2f, [512]]                          # 20 (P4 head)

  # Down to P5
  - [-1, 1, Conv, [512, 3, 2]]                   # 21
  - [[-1, 9], 1, Concat, [1]]                    # cat head P5
  - [-1, 6, C2f, [1024]]                         # 23 (P5 head)

  # Main multi-scale detect head (standard YOLOv8-style)
  - [[15, 20, 23], 1, Detect, [nc]]

  # Extra small-object detect head on refined P3 only
  - [[17], 1, Detect, [nc]]
""")

with open(MODEL_YAML, "w") as f:
    f.write(yolov8l_sea_text)

print("Saved custom model YAML to:", MODEL_YAML)

Saved custom model YAML to: /kaggle/working/cfg/yolov8_sea.yaml


In [9]:
# Create model from YOUR custom YAML (with 2 heads)
model = YOLO("/kaggle/working/cfg/yolov8_sea.yaml")

# Load pretrained weights INTO your custom architecture (partial transfer)
model.load("/kaggle/input/enhanced-yolov8-l-on-seadronessee/pytorch/v1/5/last.pt")

# Train with maritime args (starts from epoch 1 but with pretrained init)
results = model.train(
    data="/kaggle/working/cfg/seadronessee.yaml",
    epochs=15,  # total epochs
    imgsz=640,
    batch=8,
    workers=2,
    project="runs_sea",
    name="yolov8l_sea_smallrefine_v3",
    hsv_h=0.015, hsv_s=0.9, hsv_v=0.7,
    perspective=0.0005, fliplr=0.5, flipud=0.0,
    mosaic=1.0, mixup=0.15,
    device=0,
    resume=True,
    pretrained=True,  # fine-tunes from loaded weights
)

WARNING ⚠️ no model scale passed. Assuming scale='l'.
Transferred 996/996 items from pretrained weights
Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/cfg/seadronessee.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.9, hsv_v=0.7, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.15, mode=train, model=/kaggle/working/cfg/yolov8_sea.yaml, momentum=0.937, mosaic=1.0, multi_scale=Fal

In [10]:
# Validate
model.val(data=DATA_YAML, imgsz=640)

Ultralytics 8.3.241 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8_sea summary (fused): 181 layers, 72,958,380 parameters, 0 gradients, 304.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2926.8±1271.7 MB/s, size: 451.2 KB)
val: Scanning /kaggle/working/seadronessee_dataset/labels/val.cache... 4782 images, 5 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 4782/4782 9.1Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 299/299 2.9it/s 1:44
                   all       4782      25028       0.89      0.823      0.883      0.545
               swimmer       2911       6189       0.89      0.733      0.836      0.479
swimmer with life jacket       3811       9692      0.848      0.816       0.87      0.442
                  boat       3425       9147       0.93      0.922      0.943      0.714
Speed: 0.5ms preprocess, 17.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7edb559f2a80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04